In [2]:
from langchain_ollama.llms import OllamaLLM
from langchain_ollama import OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain_elasticsearch.vectorstores import ElasticsearchStore
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.chains import LLMChain
import consulta_processo as cp

model_id = "llama3.1:8b-instruct-q4_0"

embeddings = OllamaEmbeddings(model=model_id)

store = ElasticsearchStore(
    embedding=embeddings,
    index_name="processos_index",
    es_cloud_id="",
    es_api_key="",
)


llm = OllamaLLM(model=model_id, top_k=50, top_p=0.5, temperature=0.3, embed_model=embeddings)

#index = VectorStoreIndexWrapper(vectorstore=store)


In [4]:

processo_numero = "1005493-05.2019.8.26.0024"
processo = cp.searchProcesso(processo_numero)
informacoes, movimentacoes = cp.formatResult(processo)



In [ ]:


prompt_template = ("""
Como uma IA assistente você responde perguntas utilizando o contexto que vou enviar a seguir.

Context: {context}

No contexto contém um texto que é um relatório de um processo que foi extraído do site do Tribunal de Justiça.
O texto é dividido em duas partes, a primeira com metadados e informações sobre processo: juíz do processo, 
 foro do processo, vara do processo, classe do processo, assunto do processo, partes envolvidas do processo, 
histórico de movimentações ao longo do processo. Na segunda parte tem o histórico de decisões judiciais e movimentações do processo ao longo do tempo.
Memorize os nomes, locais e situações informados no texto.

A sua tarefa é responder a pergunta do usuário que vou inserir a seguir com informações do contexto.

Question: {question}


Responda com poucas palavras e de forma objetiva, utilize apenas as informações fornecidas no contexto e não invente uma resposta.
Você não deve utilizar nenhum outro conhecimento ou informação prévia.

Answer:"
""")

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain = LLMChain(
    llm=llm,
    prompt=PROMPT,
)





In [8]:
cp.printResult(processo)


# Informações e detalhes sobre sobre o processo #:
Classe do processo: Monitória
Assunto do processo: Compra e Venda
Foro do processo: Foro de Andradina
Vara do processo: 1ª Vara
Juiz do processo: Paulo Victor Alvares Gonçalves
Partes envolvidas: Reqte  Unimil Indústria e Comércio de Peças e Máquinas Agrícolas Ltda
Advogado:  Guilherme Plaça Pinto  
Reqdo  Gildo J. Vianna Transportes Eirelli - Me
Advogado:  Fernando França Teixeira de Freitas  


# Histórico de movimentações do processo #:

Sequência #0: 12/11/2019 Distribuído Livremente (por Sorteio) (movimentação exclusiva do distribuidor)
Sequência #1: 13/11/2019 Conclusos para Decisão
Sequência #2: 13/11/2019 Decisão
Vistos. Trata-se de processo no formato digital, ou seja, sem autos físicos, o qual as partes e seus advogados podem ter acesso por meio da internet pelo site (https://esaj.tjsp.jus.br/) em todas as suas movimentações. Nos termos do art. 701, do NCPC, é evidente o direito do autor, verificado por meio da prova escrita

In [10]:
def formatOutputData(chain, informacoes, movimentacoes):
    output = {
        "transito_julgado":"",
        "processo_assunto":"",
        "processo_juiz":"",
        "processo_foro":"",
        "processo_vara":"",
        "processo_classe":"",
        "processo_requerentes":[],
        "processo_requerentes_advogados":[],
        "processo_requeridos":[],
        "processo_requeridos_advogados":[],
        "ultimo_evento":"",
        "primeiro_evento":"",
        "resumo_processo":""
    }
    
    # Requerentes
    question = """
    No texto é mencionado os nomes das partes envolvidas, encontre o nome do requerente e me responda com o nome que encontrou
    """
    result = chain({"context": informacoes, "question": question} )
    output['processo_requerentes'].append(result['text'])

    # Requerentes advogados
    question = """
    No texto é mencionado os nomes das partes envolvidas, encontre o nome do advogado do requerente e me responda com o nome que encontrou
    """
    result = chain({"context": informacoes, "question": question} )
    output['processo_requerentes_advogados'].append(result['text'])

    # Requeridos
    question = """
    No texto é mencionado os nomes das partes envolvidas, encontre o nome do requerido e me responda com o nome que encontrou
    """
    result = chain({"context": informacoes, "question": question} )
    output['processo_requeridos'].append(result['text'])

    # Requeridos advogados
    question = """
    No texto é mencionado os nomes das partes envolvidas, encontre o nome do advogado do requerido e me responda com o nome que encontrou
    """
    result = chain({"context": informacoes, "question": question} )
    output['processo_requeridos_advogados'].append(result['text'])
    
    # Foro
    question = """
    No texto é mencionado o foro do processo, encontre o nome do foro e me responda com o nome que encontrou
    """
    result = chain({"context": informacoes, "question": question} )
    output['processo_foro'] = result['text']

    # Juiz
    question = """
    No texto é mencionado o juíz do processo, encontre o nome do juíz e responda com o nome que encontrou
    """
    result = chain({"context": informacoes, "question": question} )
    output['processo_juiz'] = result['text']

    # Vara
    question = """
    No texto é mencionado a vara do processo, encontre o nome da vara e me responda com o nome que encontrou
    """
    result = chain({"context": informacoes, "question": question} )
    output['processo_vara'] = result['text']

    # Classe
    question = """
    No texto é mencionado a classe do processo, encontre a classe e me responda com o nome que encontrou
    """
    result = chain({"context": informacoes, "question": question} )
    output['processo_classe'] = result['text']

    # Assunto
    question = """
    No texto é mencionado o assunto do processo, encontre o assunto e me responda com o nome que encontrou
    """
    result = chain({"context": informacoes, "question": question} )
    output['processo_assunto'] = result['text']

    # Transito em julgado
    question = """
    Procure no no histórico de movimentações algum dos termos similares a  Arquivado Definitivamente ou cumprimento de setença, memorize essa informação. 
    Se você encontrou os termos, a sua resposta vai ser que foi transitado em julgado. 
    Caso contrário, se não encontrou os termos, então vai responder que não transitou em julgado.
    """
    result = chain({"context": movimentacoes, "question": question} )
    output['transito_julgado'] = result['text']

    # Ultimo evento
    question = """
    Procure pelo histórico de movimentações que está ordenado  de forma que a movimentação mais antiga (sequência 0),
    avançando para a movimentação mais recente (sequência 0+n).
    A sua tarefa é responder com a movimentação mais recente (sequências de n - 1) que você localizou.
    """
    result = chain({"context": movimentacoes, "question": question} )
    output['ultimo_evento'] = result['text']

    # Resumo evento
    question = """
    No texto contém o histórico de movimentações que está ordenado de forma que a movimentação mais antiga (sequência 0),
    avançando para a movimentação mais recente (sequência 0+n).
    A sua tarefa é identificar as movimentações mais recentes (sequências de n - 3) e responder com o resumo bem explicativo sobre as decisões no processo.
    """
    result = chain({"context": movimentacoes, "question": question} )
    output['resumo_processo'] = result['text']

    return output

In [ ]:


output = formatOutputData(chain, informacoes, movimentacoes)

output